### Read the HLT-Report 
*HLTReport*
  * path name
    * WasRun
    * L1S
    * Pre
    * HLT
    * %L1sPre
    * Rate
    * RateHi
    * Errors

In [100]:
def getHLTReport(fname):
    f=open('logNoCust','r')
    l=f.readline()
    hltReportDict={'WasRun':0.0,'L1S':0.0,'Pre':0.0,'HLT':0.0,'%L1sPre':0,'Rate':0.0,'RateHi':0.0, 'Errors':0}
    hltReportKeys=list(hltReportDict.keys())
    HLTReport={}
    SummaryFlag=False
    while l:
        if 'HLT-Report' not in l:
            l=f.readline()
            continue
        if '---------- HLTrig Summary ------------' in l:
            SummaryFlag=True
            l=f.readline()
            l=f.readline()
            continue
        if 'end!' in l:
            SummaryFlag=False
            break
        if SummaryFlag:
            currdict=hltReportDict.copy()
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
            items.pop(0)
            for i in range(len(hltReportKeys)):
                currdict[hltReportKeys[i]]=float(items[i])
            HLTReport.update({items[-1]:currdict})
        l=f.readline()
    f.close()
    return HLTReport

### Read the Time Report
 
*TimeReport*
  * pathSummary
    * perEvent
    * perExec
    
  * pathDetails 
    * path_name
      * perEvent 
      * perVisit
      * Name
      
  * moduleSummary 
    * module_name
      * perEvent
      * perExec
      * perVisit  
    

In [102]:
def getTimeReport(fname):
    f=open(fname,'r')
    l=f.readline()

    timeReportDict={'perEvent':0.0,'perExec':0.0}
    timeReportKeys=list(timeReportDict.keys())

    timeReportDetailsDict={'perEvent':[],'perVisit':[],'Name':[]}
    timeReportDetailsDictKeys=list(timeReportDetailsDict.keys())

    timeReportModulesDict={'perEvent':[],'perExec':[],'perVisit':[]}
    timeReportModulesDictKeys=list(timeReportModulesDict.keys())

    TimeReport={'pathSummary':{},'pathDetails':{},'moduleSummary':{}}
    SummaryFlag=False
    DetailsFlag=False
    ModulesFlag=False
    while l:
        if 'TimeReport' not in l:
            SummaryFlag=False
            DetailsFlag=False
            ModulesFlag=False
            l=f.readline()
            continue
        if 'per event    per visit  Name' in l :
            l=f.readline()
            continue
    #     print(l[:-1])
        if '---------- Path   Summary ---' in l:
            SummaryFlag=True
            l=f.readline()
            l=f.readline()
            continue
        if '---------- Module Summary --' in l:
            ModulesFlag=True
            l=f.readline()
            l=f.readline()
            continue

        if '---------- Modules in Path:' in l:
            DetailsFlag=True
            l=l[:-1].replace("---------- Modules in Path: ","")
            l=l.replace(" ---[Real sec]----","")
            currPath=l.replace("TimeReport ","")
            TimeReport['pathDetails'].update({currPath:timeReportDetailsDict.copy()})
    #         print(l)
            l=f.readline()
            l=f.readline()
            continue

        ## bug fix
        if 'per event ' in l:
            SummaryFlag=False
            ModulesFlag=False

        if SummaryFlag:
            currdict=timeReportDict.copy()
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
    #         print(items)
            for i in range(len(timeReportKeys)):
                currdict[timeReportKeys[i]]=float(items[i])
            TimeReport['pathSummary'].update({items[-1]:currdict})

        if DetailsFlag:
    #         print(l[:-1])
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
            for i in range(len(timeReportDetailsDictKeys)-1):
                TimeReport['pathDetails'][currPath][timeReportDetailsDictKeys[i]].append(float(items[i]))
            TimeReport['pathDetails'][currPath][timeReportDetailsDictKeys[-1]].append(items[i])

        if ModulesFlag:
            currdict=timeReportModulesDict.copy()
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
    #         print(l[:-1])
    #         print(items)
            for i in range(len(timeReportModulesDictKeys)):
                currdict[timeReportModulesDictKeys[i]]=float(items[i])
            TimeReport['moduleSummary'].update({items[-1]:currdict})

        l=f.readline()
    f.close()
    return TimeReport

In [101]:
HLTReport=getHLTReport('logNoCust')
TimeReport=getTimeReport('logNoCust')
print(" total number of  paths  = ",len(HLTReport))
print(" total number of  paths in pathSummary  = ",len(TimeReport['pathSummary']))
print(" total number of  paths in pathDetails  = ",len(TimeReport['pathDetails']))
print(" total number of  modules in moduleSummary  = ",len(TimeReport['moduleSummary']))

 total number of  paths  =  739
 total number of  paths in pathSummary  =  739
 total number of  paths in pathDetails  =  739
 total number of  modules in moduleSummary  =  6367
